In [1]:
import pandas as pd
import os
import shutil
import subprocess

In [2]:
# output='/node05/program/jiachenghao/Mycoplasmoides_pneumoniae/Qulity/mNGS/'
# input_path='/node05/program/jiachenghao/Mycoplasmoides_pneumoniae/肺炎支原体测序数据/Transfer/'
# input_path_list=os.listdir(input_path)
# file=pd.read_excel('QC.xlsx')['File'].to_list()

# for i in file:
#     copy_file=[x for x in input_path_list if x.startswith(i)]
#     for ii in copy_file:
#          shutil.copy(input_path+ii,output)
    # if len(copy_file) == 1:
    #     copy_file=copy_file[0]
    # shutil.copy(input_path+copy_file,output)

In [3]:
def Cal_overage(genome_coverage_txt):
    global coverage_percentage
    coverage_data = []
    with open('genome_coverage.txt', 'r') as file:
        for line in file:
            parts = line.strip().split()
            reference, depth, num_bases, total_bases, fraction = parts
            if reference == 'genome':
                depth = int(depth)
                num_bases = int(num_bases)
                total_bases = int(total_bases)
                fraction = float(fraction)
                coverage_data.append((depth, num_bases, total_bases, fraction))
    total_bases = coverage_data[0][2]
    total_coverage = sum(depth * num_bases for depth, num_bases, _, _ in coverage_data)
    average_coverage = total_coverage / total_bases
    # print(f"Average Coverage: {average_coverage:.2f}")
    covered_bases = total_bases - next(num_bases for depth, num_bases, _, _ in coverage_data if depth == 0)
    coverage_percentage = (covered_bases / total_bases) * 100
    # print(f"Coverage Percentage for : {coverage_percentage:.2f}%")


In [4]:
output='/node05/program/jiachenghao/Mycoplasmoides_pneumoniae/Qulity/test/'   ######
Script_path='/node05/program/jiachenghao/Mycoplasmoides_pneumoniae/Qulity/'
workpath='/node05/program/jiachenghao/Mycoplasmoides_pneumoniae/Qulity/coverage/'
output_list=os.listdir(output)

In [5]:
Process_file=os.listdir(output)
Single_list=[]
for i in Process_file:
    i=i.split('_')[2]
    Single_list.append(i)
Single_list=list(set(Single_list))
Single_list

['UDB-143']

In [6]:
File=[]
Coverage=[]
for i in Single_list:
    File.append(i)
    Check=os.listdir(workpath)
    if len(Check) != 0:
        print('Folder not Empty')
    print('Start Process ', i)
    
    index1=i+'_1'
    index2=i+'_2'
    file1=[x for x in output_list if str(index1) in x][0]
    file2=[x for x in output_list if str(index2) in x][0]

    shutil.copy(output+file1,workpath)
    shutil.copy(output+file2,workpath)
    shutil.copy(Script_path+'NC_000912.1.fasta',workpath)
    shutil.copy(Script_path+'step2.sh',workpath)
    shutil.copy(Script_path+'step3.sh',workpath)
##################################################################
    command = ["bowtie2-build", "NC_000912.1.fasta", "refgenome"]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=workpath)
    if result.returncode == 0:
        print("Bowtie2 DB Run Success")
        print(result.stdout.decode())
    else:
        print("Bowtie2 DB Run Failed")
        print(result.stderr.decode())
################################################################
    command = ["sh", "step2.sh"]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=workpath)
    if result.returncode == 0:
        print("Step2 Run Success")
        print(result.stdout.decode())
    else:
        print("Step2 Run Failed")
        print(result.stderr.decode())
###############################################################
    if os.path.isfile(workpath+'mapping_result_sorted.bam'):
        print('bam file is ready')
    else:
        print('Error')
###############################################################
    command = ["sh", "step3.sh"]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=workpath)
    if result.returncode == 0:
        print("Step3 Run Success")
        print(result.stdout.decode())
    else:
        print("Step3 Run Failed")
        print(result.stderr.decode())
###############################################################
    if os.path.isfile(workpath+'genome_coverage.txt'):
        print('Genome_coverage file is ready')
    else:
        print('Error')


    shutil.copy(workpath+'genome_coverage.txt',Script_path)
    Cal_overage('genome_coverage.txt')
    Coverage.append(coverage_percentage)

    os.remove(Script_path+'genome_coverage.txt')
    delet=os.listdir(workpath)
    for i in delet:
        os.remove(workpath+i)

Start Process  UDB-143
Bowtie2 DB Run Success
Settings:
  Output files: "refgenome.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  NC_000912.1.fasta
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 204098
Using parameters --bmax 153074 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 153074 --dcv 1024
Cons

In [7]:
File

['UDB-143']

In [8]:
Coverage

[94.14265660943123]

In [9]:
out={'File':File,'Coverage':Coverage}
out=pd.DataFrame(out)
out.to_csv('mNGS_Coverage.csv')